In [1]:
from urllib.request import Request,urlopen
import requests
from bs4 import BeautifulSoup as soup
from time import sleep
from io import StringIO
import re
from unidecode import unidecode
import os
import pybaseball as bb
import pandas as pd
import numpy as np

import BaseballScrapingModule

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

import random
import warnings
warnings.filterwarnings("ignore", message="The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.")
warnings.filterwarnings("ignore", message="invalid value encountered in cast")
pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [ ]:
data_path = r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\Data"
# data_path = r"C:\Users\jmiller\Downloads"

In [3]:
test = pd.read_csv(r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\bullpens.csv")
test

,Season,End Date,Team,ERA,WAR,K/9,SV,ERA_norm,K9_norm,SV_norm,WAR_norm,Bullpen_Rating
0,2024,2024-07-10,CLE,2.60,4.6,9.65,33,1.000000,0.793220,0.947368,0.869565,0.9025
1,2024,2024-07-10,PHI,3.36,5.5,10.23,26,0.766871,0.989831,0.578947,1.000000,0.8339
2,2024,2024-07-10,MIN,3.56,3.2,9.44,28,0.705521,0.722034,0.684211,0.666667,0.6946
3,2024,2024-07-10,BAL,3.79,3.0,8.93,31,0.634969,0.549153,0.842105,0.637681,0.6660
4,2024,2024-07-10,STL,3.50,2.6,8.23,34,0.723926,0.311864,1.000000,0.579710,0.6539
...,...,...,...,...,...,...,...,...,...,...,...,...
9747,2024,2024-05-17,MIA,4.61,1.0,8.46,7,0.223827,0.357333,0.000000,0.465116,0.2616
9748,2024,2024-05-17,LAA,5.22,0.2,9.19,8,0.003610,0.552000,0.090909,0.279070,0.2314
9749,2024,2024-05-17,ARI,4.47,0.6,7.47,9,0.274368,0.093333,0.181818,0.372093,0.2304
9750,2024,2024-05-17,TOR,4.92,-0.6,7.87,11,0.111913,0.200000,0.363636,0.093023,0.1921


In [4]:
test1 = pd.read_csv(r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\team_gamelogs.csv")
test1

,Game,Date,Team,Home,Opp,Rslt,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,ROE,GDP,SB,CS,BA,OBP,SLG,OPS,LOB,NumPlayers,OppStart
0,1,2023-03-30,KCR,True,MIN,"L,0-2",33,27,0,2,1,0,0,0,5,0,11,1,0,0,0,2,0,0,0.074,0.242,0.111,0.354,6,9,P.Lopez
1,2,2023-04-01,KCR,True,MIN,"L,0-2",35,30,0,4,1,0,0,0,5,0,5,0,0,0,1,2,0,0,0.105,0.250,0.140,0.390,8,10,S.Gray
2,3,2023-04-02,KCR,True,MIN,"L,4-7",36,33,4,6,2,0,2,4,3,0,9,0,0,0,0,0,0,0,0.133,0.250,0.244,0.494,5,9,J.Ryan
3,4,2023-04-03,KCR,True,TOR,"W,9-5",37,33,9,10,2,1,1,9,3,0,11,1,0,0,0,0,2,0,0.179,0.284,0.317,0.601,4,9,J.Berrios
4,5,2023-04-04,KCR,True,TOR,"L,1-4",31,30,1,4,0,0,1,1,1,0,7,0,0,0,0,0,0,0,0.170,0.262,0.301,0.562,3,10,Y.Kikuchi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9713,158,2024-09-25,MIA,False,MIN,"L,3-8",39,38,3,11,2,0,1,3,1,0,16,0,0,0,0,0,0,0,0.242,0.298,0.375,0.672,9,9,S.Woods Richardson
9714,159,2024-09-26,MIA,False,MIN,"W,8-6",58,51,8,13,4,0,0,7,6,1,20,0,0,1,0,0,3,1,0.242,0.298,0.374,0.672,15,14,D.Festa
9715,160,2024-09-27,MIA,False,TOR,"W,15-5",48,40,15,17,2,3,3,14,7,0,8,0,1,0,0,1,0,0,0.243,0.300,0.378,0.677,6,11,J.Berrios
9716,161,2024-09-28,MIA,False,TOR,"W,8-1",39,37,8,11,3,0,1,8,2,0,8,0,0,0,0,0,1,0,0.244,0.300,0.378,0.678,4,9,Y.Rodriguez


In [8]:
needed_dates = list(set(test1['Date'].to_list()).difference(set(test['End Date'].to_list())))
needed_dates

['2024-04-01',
 '2024-03-28',
 '2024-09-13',
 '2023-07-14',
 '2023-04-08',
 '2023-04-05',
 '2023-07-26',
 '2023-05-10',
 '2024-09-30',
 '2024-08-11',
 '2024-05-04',
 '2023-05-20',
 '2024-08-27',
 '2024-05-15',
 '2024-06-11',
 '2023-06-29',
 '2024-03-20',
 '2024-05-21',
 '2024-07-29',
 '2024-06-29',
 '2024-08-14',
 '2024-07-19',
 '2023-05-19',
 '2023-08-29',
 '2024-04-19',
 '2024-04-09',
 '2023-09-21',
 '2024-06-30',
 '2024-07-01',
 '2024-07-26',
 '2024-05-10',
 '2024-08-24',
 '2023-07-28',
 '2024-09-03',
 '2023-05-23',
 '2023-04-25',
 '2024-06-04',
 '2023-08-09',
 '2023-05-04',
 '2024-06-05',
 '2023-05-14']

In [11]:
from BaseballScrapingModule import BaseballScraper

data_pull = BaseballScraper()

df = pd.DataFrame()
for game_date in pd.to_datetime(needed_dates):
    temp = data_pull.rankBullpens(game_date.year, game_date)
    df = pd.concat([df, temp])
if 'bullpens.csv' in os.listdir(data_path):
    old_df = pd.read_csv(os.path.join(data_path, 'bullpens.csv'))
    df = pd.concat([old_df, df]).drop_duplicates(subset=['Team', 'Season','End Date'], keep='last')
    df['End Date'] = pd.to_datetime(df['End Date'])
    df.sort_values('End Date', inplace=True)
df.to_csv(os.path.join(data_path, 'bullpens.csv'), index=False)

Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for

In [3]:
abr_convert = teams_abbr = {"AZ" : 'ARI',
    "ARI": 'ARI',
    "LAD" : 'LAD',
    "SD": 'SDP',
    "SF": 'SFG',
    "COL": 'COL',
    "HOU" : 'HOU',
    "TEX" : 'TEX',
    "SEA": 'SEA',
    "LAA": 'LAA',
    "OAK": 'OAK',
    "MIL": 'MIL',
    "CHC": 'CHC',
    'Chi. Cubs': 'CHC',
    "CIN": 'CIN',
    "PIT": 'PIT',
    "STL": 'STL',
    "MIN": 'MIN',
    "DET": 'DET',
    'Detroit': 'DET',
    "CLE": 'CLE',
    "CWS": 'CHW',
    'CHW':'CHW',
    'Chi. White Sox' : 'CHW',
    "KC": 'KCR',
    "ATL": 'ATL',
    "PHI": 'PHI',
    "MIA": 'MIA',
    'Miami': 'MIA',
    "NYM": 'NYM',
    "WSH": 'WSN',
    'Washington':'WSN',
    'WAS':'WSN',
    "BAL": 'BAL',
    "TB": 'TBR',
    "TOR": 'TOR',
    "NYY": 'NYY',
    "BOS": 'BOS',
    'Boston':'BOS'}

In [4]:
### USED TO CREATE CSV THAT CONTAINS PITCHER CATEGORIES ###
def categorizePitchers():
    ### GET PITCHERS FROM DESIRED SEASONS ###
    pitchers = bb.fg_pitching_data(2023, 2024, qual=10)
    ### GET IDS FOR ALL THOSE PITCHERS ###
    pitchers_ids = bb.playerid_reverse_lookup(pitchers['IDfg'], key_type='fangraphs')

    names, mlbIDs, bbrefIDs, fgIDs, years = [], [], [], [], []
    final_stats = []
    for i in range(len(pitchers_ids)):
        print('Pitcher {0} of {1}'.format(i, len(pitchers_ids)), end='\n')
        ### GRAB DIFFERENT SEASONS FOR A PITCHER (ACCOUNT FOR THE CHANGE THEY MAY HAVE THROUGH SEASONS) ###
        start_year, end_year = int(pitchers_ids.iloc[i, 6]), int(pitchers_ids.iloc[i, 7])+1
        ### MAKE SURE THE DATA WE GET IS JUST 2021+ ###
        if start_year <= 2022: start_year = 2023
        for year in range(start_year, end_year):
            try:
                ### GRABBING SPLITS MAY ERROR, SKIP THOSE WHO DO ###
                pitcher_splits = bb.get_splits(playerid=pitchers_ids.iloc[i, 4], year=year, pitching_splits=True, player_info=True)
                ##########################################################################
                ### CATEGORIZATION BASED ON SPLITS/PITCH SPEED AND SPIN ##################
                ##########################################################################
                ### GRAB WHAT HANDEDNESS THE PITCHER IS ###
                if pitcher_splits[1]['Throws'] == 'Right ':
                    stats = np.array([0])
                elif pitcher_splits[1]['Throws'] == 'Left ':
                    stats = np.array([1])
                else:
                    stats = np.array([2])
                ### APPEND ROWS OF INTEREST TO OUR STATS ###
                stats = np.append(stats, [pitcher_splits[0].loc['Platoon Splits', 'vs LHB'].to_numpy(),
                                            pitcher_splits[0].loc['Platoon Splits', 'vs RHB'].to_numpy(),
                                            pitcher_splits[0].loc['Home or Away', 'Home'].to_numpy(),
                                            pitcher_splits[0].loc['Home or Away', 'Away'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Ground Balls'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Fly Balls'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Line Drives'].to_numpy(),
                                            pitcher_splits[0].loc['Game Conditions', 'Day'].to_numpy(),
                                            pitcher_splits[0].loc['Game Conditions', 'Night'].to_numpy()])
                ### GET AVG SPIN AND AVG SPEED OF PITCH TYPES ###
                pitch_spin = bb.statcast_pitcher_pitch_arsenal(year=year, minP=0, arsenal_type='avg_spin')
                stats = np.append(stats, pitch_spin.loc[pitch_spin['pitcher']==pitchers_ids.iloc[i, 2], pitch_spin.columns[2:]].to_numpy())

                pitch_speed = bb.statcast_pitcher_pitch_arsenal(year=year, minP=0, arsenal_type='avg_speed')
                stats = np.append(stats, pitch_speed.loc[pitch_speed['pitcher']==pitchers_ids.iloc[i, 2], pitch_speed.columns[2:]].to_numpy())

                stats = pd.Series(stats).replace(pd.NA, np.nan).to_numpy()

                if len(np.nan_to_num(stats))==273:
                    ### FOR FUTURE CREATING OF PITCHER CATEGORY TABLE ###
                    final_stats.append(np.nan_to_num(stats))
                    names.append(pitchers_ids.iloc[i, 0]+', '+pitchers_ids.iloc[i, 1])
                    mlbIDs.append(pitchers_ids.iloc[i, 2])
                    bbrefIDs.append(pitchers_ids.iloc[i, 4])
                    fgIDs.append(pitchers_ids.iloc[i, 5])
                    years.append(year)
                else:
                    print('Pitcher {0} only had {1} data columns for season {2}, skipping.'.format(pitchers_ids.iloc[i,4], len(np.nan_to_num(stats)), year))
            except:
                print('Pitcher {0} encountered errors for season {1}, skipping.'.format(pitchers_ids.iloc[i, 4], year))

    final_stats = np.array(final_stats)
    kmeans = KMeans().fit(final_stats)
    pitcher_categories = pd.DataFrame({'Pitcher':names, 'Season':years, 'mlbID':mlbIDs, 'bbrefID':bbrefIDs, 'fgID':fgIDs, 'Categorization': kmeans.labels_})
    pitcher_categories['Pitcher'] = pitcher_categories['Pitcher'].apply(unidecode)

    return pitcher_categories
    # pitcher_categories.to_csv(os.path.join(data_path, "pitcher_categories.csv"), index=False)

In [5]:
# categorizePitchers()

In [ ]:
all_teams = set(teams_abbr.values())
all_team_gamelogs_batting = pd.DataFrame()

for season in [2023, 2024]:
    for team in all_teams:
        gamelog_batting = bb.team_game_logs(season, team, 'batting')
        gamelog_batting['Date'] = pd.to_datetime(gamelog_batting['Date'].str[:6].str.strip() + ', {0}'.format(season))
        gamelog_batting['OppStart'] = gamelog_batting['OppStart'].str.split('(').str[0]
        gamelog_batting['OppStart'] = gamelog_batting['OppStart'].apply(unidecode)
        gamelog_batting.insert(2, 'Team', team)
        all_team_gamelogs_batting = pd.concat([all_team_gamelogs_batting, gamelog_batting.drop('Thr', axis=1)])
        
all_team_gamelogs_batting.to_csv(os.path.join(data_path, "team_gamelogs.csv"), index=False)

In [5]:
def structureGames():
    combined_gamelog = pd.read_csv(os.path.join(data_path, "team_gamelogs.csv"))
    combined_gamelog['Date'] = pd.to_datetime(combined_gamelog['Date'])
    combined_gamelog['Season'] = combined_gamelog['Date'].dt.year
    combined_gamelog = combined_gamelog.sort_values('Date')
    combined_gamelog['DoubleHeader'] = combined_gamelog[['Date', 'Team', 'Opp']].duplicated()
    combined_gamelog['1B'] = combined_gamelog['H'] - (combined_gamelog['2B']+combined_gamelog['3B']+combined_gamelog['HR'])
    # combined_gamelog.rename({'H':'1B'}, axis=1, inplace=True)

    roster = pd.read_csv(os.path.join(data_path, "rosters.csv")).rename({'bbref_id':'bbrefID', 'Name':'Pitcher'}, axis=1)
    pitcher_cat = pd.read_csv(os.path.join(data_path, "pitcher_categories.csv"), usecols=['Season', 'bbrefID', 'Categorization'])
    pitcher_cat = pd.merge(pitcher_cat, roster, 'left', ['Season', 'bbrefID']).rename({'Team':'Opp'}, axis=1)
    pitcher_cat['OppStart'] = pitcher_cat['Pitcher'].str.split(' ').str[0].str[0] + '.' +  pitcher_cat['Pitcher'].str.split(' ').str[1]
    pitcher_cat = pitcher_cat[['OppStart', 'Season', 'Opp', 'Categorization']]

    combined_gamelog = pd.merge(combined_gamelog, pitcher_cat, 'left', ['OppStart', 'Season', 'Opp'])

    home_teams, away_teams = combined_gamelog.loc[combined_gamelog['Home']==True], combined_gamelog.loc[combined_gamelog['Home']!=True]
    combined_gamelog = pd.merge(home_teams, away_teams, 'inner', left_on=['Date','Team','Opp', 'DoubleHeader'], right_on=['Date','Opp','Team', 'DoubleHeader'],suffixes=['_home', '_away'])
    combined_gamelog = combined_gamelog.drop(['Game_home', 'Home_home', 'Opp_home', 'Rslt_home', 'BA_home', 'OBP_home', 'SLG_home', 'OPS_home', 'NumPlayers_home', 'Season_home', 'DoubleHeader',
                                            'Game_away', 'Home_away', 'Opp_away', 'Rslt_away', 'BA_away', 'OBP_away', 'SLG_away', 'OPS_away', 'NumPlayers_away', 'Season_away'], axis=1)
    combined_gamelog['Result'] = (combined_gamelog['R_home']>combined_gamelog['R_away']).astype(int)
    combined_gamelog = combined_gamelog.dropna(subset=['Categorization_home', 'Categorization_away']).reset_index(drop=True)
    return combined_gamelog

In [13]:
### USED TO GET ALL ARRAYS UP TO A GIVEN DATE. USEFUL FOR SIMULATING AND GRABBING THE ARRAYS UP TO A GAME DAY TO TRAIN ON BEFORE PREDICTING ###
# col_home =['Team_home','PA_home','AB_home','R_home','H_home','1B_home','2B_home','3B_home','HR_home','BB_home','SO_home','HBP_home',
#                                     'SH_home','ROE_home','SB_home','CS_home','LOB_home']
# col_away = ['Team_away','PA_away','AB_away','R_away','H_away','1B_away','2B_away','3B_away','HR_away','BB_away','SO_away','HBP_away',
#                                     'SH_away','ROE_away','SB_away','CS_away','LOB_away']
col_home =['Team_home','PA_home','AB_home','R_home','H_home','1B_home','2B_home','3B_home','HR_home','BB_home','SO_home','HBP_home',
                                    'SH_home','SB_home','CS_home','LOB_home']
col_away = ['Team_away','PA_away','AB_away','R_away','H_away','1B_away','2B_away','3B_away','HR_away','BB_away','SO_away','HBP_away',
                                    'SH_away','SB_away','CS_away','LOB_away']
def getArraysUpToDate(odds, gamelogs, current_game_date, data_pull):

    gamelogs = gamelogs[gamelogs['Date']<current_game_date]
    # gamelogs = gamelogs.loc[gamelogs['Date'].dt.year==2023]
    game_stats, game_results, all_home_odds, all_away_odds, all_home_teams, all_away_teams, dates = [], [], [], [], [], [], []
    for row in range(len(gamelogs)):
        ### GETTING HOME TEAM LAST GAMES STATS ###
        date, team, opp, pitcher, pitcher_cat, runs_home, runs_away = gamelogs.loc[row, ['Date', 'Team_home', 'Team_away', 'OppStart_home', 'Categorization_home', 'R_home', 'R_away']]
        ### GET BULLPEN RANKING UP TO DATE ###
        bullpens = data_pull.rankBullpens(date.year, date)
        ### GET GAME RESULT ### 
        game_outcome = (runs_home>runs_away).astype(int)
        ### GET GAME ODDS ###
        try:
            home_odds = odds.loc[(odds['Date']==date)&(odds['Home Team']==team)&(odds['Away Team']==opp), 'Home W Odds'].to_list()[0]
            away_odds = odds.loc[(odds['Date']==date)&(odds['Home Team']==team)&(odds['Away Team']==opp), 'Away W Odds'].to_list()[0]
        except:
            print('No odds for game on ', date, ' for ', team, ' - ', opp)
        ### GET GAMES TEAM HAD AGAINST SPECIFIC PITCHER THIS SEASON ###
        against_pitcher_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['OppStart_home']==pitcher)), 
                                col_home]

        against_pitcher_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['OppStart_away']==pitcher)&(gamelogs['Team_away']==team)), 
                                col_away]
        against_pitcher_away.columns = col_home
        against_pitcher = pd.concat([against_pitcher_away, against_pitcher_home])
        ### GET GAMES TEAM HAD AGAINST PITCHER CATEGORY THIS SEASON ###
        against_pitcher_cat_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Categorization_home']==pitcher_cat)&(gamelogs['OppStart_home']!=pitcher)),
                                    col_home]
        
        against_pitcher_cat_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Categorization_away']==pitcher_cat)&(gamelogs['OppStart_away']!=pitcher)&(gamelogs['Team_away']==team)),
                                    col_away]
        against_pitcher_cat_away.columns = col_home
        against_pitcher_cat = pd.concat([against_pitcher_cat_away, against_pitcher_cat_home])
        # against_pitcher_cat = against_pitcher_cat[-5:]
        # against_pitcher_cat = pd.DataFrame()
        ### GET GAMES TEAM HAD AGAINST OPPOSING TEAM THIS SEASON ###
        against_team_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Team_away']==opp)&(gamelogs['OppStart_home']!=pitcher)&(gamelogs['Categorization_home']!=pitcher_cat)),
                                col_home]
        
        against_team_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&(((gamelogs['Team_home']==opp)&(gamelogs['Team_away']==team)&(gamelogs['OppStart_away']!=pitcher)&(gamelogs['Categorization_away']!=pitcher_cat))),
                                col_away]
        against_team_away.columns = col_home
        against_team = pd.concat([against_team_away, against_team_home])
        # against_team = against_team[-5:]
        
        home_team_last = []
        if len(against_pitcher)+len(against_pitcher_cat)+len(against_team)>3:
            home_team_last = pd.concat([against_pitcher, against_pitcher_cat, against_team]).groupby('Team_home', as_index=False).mean().drop('Team_home',axis=1)
            home_team_last.loc[~home_team_last['AB_home'].isna(), 'BA_home'] = np.round(home_team_last.loc[~home_team_last['AB_home'].isna(), 'H_home']/home_team_last.loc[~home_team_last['AB_home'].isna(), 'AB_home'],3)
            home_team_last.loc[~home_team_last['PA_home'].isna(), 'OBP_home'] = np.round((home_team_last.loc[~home_team_last['PA_home'].isna(), 'H_home']+home_team_last.loc[~home_team_last['PA_home'].isna(), 'BB_home'])\
                                                                                /home_team_last.loc[~home_team_last['PA_home'].isna(), 'PA_home'],3)
            home_team_last.loc[~home_team_last['AB_home'].isna(), 'SLG_home'] = np.round((home_team_last.loc[~home_team_last['AB_home'].isna(), '1B_home']+2*home_team_last.loc[~home_team_last['AB_home'].isna(), '2B_home']+\
                                                                                3*home_team_last.loc[~home_team_last['AB_home'].isna(), '3B_home']+4*home_team_last.loc[~home_team_last['AB_home'].isna(), 'HR_home'])\
                                                                                /home_team_last.loc[~home_team_last['AB_home'].isna(), 'AB_home'],3)
            home_team_last['BP_home'] = bullpens.loc[(bullpens['Team']==team)&(bullpens['Season']==date.year), 'Bullpen_Rating'].to_list()[0]

        ### GETTING AWAY TEAM LAST GAMES STATS ###
        date, team, opp, pitcher, pitcher_cat, runs_home, runs_away = gamelogs.loc[row, ['Date', 'Team_away', 'Team_home', 'OppStart_away', 'Categorization_away', 'R_home', 'R_away']]
        ### GET GAMES TEAM HAD AGAINST SPECIFIC PITCHER THIS SEASON ###
        against_pitcher_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['OppStart_home']==pitcher)), 
                                col_home]

        against_pitcher_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['OppStart_away']==pitcher)&(gamelogs['Team_away']==team)), 
                                col_away]
        against_pitcher_away.columns = col_home
        against_pitcher = pd.concat([against_pitcher_away, against_pitcher_home])
        ### GET GAMES TEAM HAD AGAINST PITCHER CATEGORY THIS SEASON ###
        against_pitcher_cat_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Categorization_home']==pitcher_cat)&(gamelogs['OppStart_home']!=pitcher)),
                                    col_home]
        
        against_pitcher_cat_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Categorization_away']==pitcher_cat)&(gamelogs['OppStart_away']!=pitcher)&(gamelogs['Team_away']==team)),
                                    col_away]
        against_pitcher_cat_away.columns = col_home
        against_pitcher_cat = pd.concat([against_pitcher_cat_away, against_pitcher_cat_home])
        # against_pitcher_cat = against_pitcher_cat[-5:]
        # against_pitcher_cat = pd.DataFrame()
        ### GET GAMES TEAM HAD AGAINST OPPOSING TEAM THIS SEASON ###
        against_team_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Team_away']==opp)&(gamelogs['OppStart_home']!=pitcher)&(gamelogs['Categorization_home']!=pitcher_cat)),
                                col_home]
        
        against_team_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&(((gamelogs['Team_home']==opp)&(gamelogs['Team_away']==team)&(gamelogs['OppStart_away']!=pitcher)&(gamelogs['Categorization_away']!=pitcher_cat))),
                                col_away]
        against_team_away.columns = col_home
        against_team = pd.concat([against_team_away, against_team_home])
        # against_team = against_team[-5:]
        
        away_team_last = []
        if len(against_pitcher)+len(against_pitcher_cat)+len(against_team)>3:
            away_team_last = pd.concat([against_pitcher, against_pitcher_cat, against_team]).groupby('Team_home', as_index=False).mean().drop('Team_home',axis=1)
            away_team_last.loc[~away_team_last['AB_home'].isna(), 'BA_home'] = np.round(away_team_last.loc[~away_team_last['AB_home'].isna(), 'H_home']/away_team_last.loc[~away_team_last['AB_home'].isna(), 'AB_home'],3)
            away_team_last.loc[~away_team_last['PA_home'].isna(), 'OBP_home'] = np.round((away_team_last.loc[~away_team_last['PA_home'].isna(), 'H_home']+away_team_last.loc[~away_team_last['PA_home'].isna(), 'BB_home'])\
                                                                                /away_team_last.loc[~away_team_last['PA_home'].isna(), 'PA_home'],3)
            away_team_last.loc[~away_team_last['AB_home'].isna(), 'SLG_home'] = np.round((away_team_last.loc[~away_team_last['AB_home'].isna(), '1B_home']+2*away_team_last.loc[~away_team_last['AB_home'].isna(), '2B_home']+\
                                                                                3*away_team_last.loc[~away_team_last['AB_home'].isna(), '3B_home']+4*away_team_last.loc[~away_team_last['AB_home'].isna(), 'HR_home'])\
                                                                                /away_team_last.loc[~away_team_last['AB_home'].isna(), 'AB_home'],3)
            away_team_last['BP_home'] = bullpens.loc[(bullpens['Team']==team)&(bullpens['Season']==date.year), 'Bullpen_Rating'].to_list()[0]
        # print(row, ':', opp, len(home_team_last), team, len(away_team_last))
        if (len(home_team_last)!=0) & (len(away_team_last)!=0):
            if (False in np.isnan(home_team_last.to_numpy())) & (False in np.isnan(away_team_last.to_numpy())) & (~np.isnan(home_odds)) & (~np.isnan(away_odds)):
                game_results.append(game_outcome)
                game_stats.append(np.round(pd.concat([home_team_last, away_team_last], axis=1),3).to_numpy())
                all_home_odds.append(home_odds)
                all_away_odds.append(away_odds)
                all_home_teams.append(opp)
                all_away_teams.append(team)
                dates.append(date)
    
    return game_stats, game_results, all_home_odds, all_away_odds, all_home_teams, all_away_teams, dates

In [14]:
### RETURNS THE AGGREGATED LAST GAME STATS FOR TEAMS PLAYING ON A GIVEN DAY. USEFUL FOR GETTING THE ARRAY IN WHICH WE'LL PREDICT HOME TEAM WINS ###
def getArraysForDate(odds, gamelogs, date, teams, opps, pitcher_homes, pitcher_cat_homes, pitcher_aways, pitcher_cat_aways, results, data_pull):

    game_stats, all_home_odds, all_away_odds, new_results = [], [], [], []
    bullpens = data_pull.rankBullpens(date.year, date)
    for i in range(len(teams)):
        team, opp, pitcher_home, pitcher_cat_home, pitcher_away, pitcher_cat_away = teams[i], opps[i], pitcher_homes[i], pitcher_cat_homes[i], pitcher_aways[i], pitcher_cat_aways[i]
        ### GETTING HOME TEAM LAST GAMES STATS ###
        #date, team, opp, pitcher, pitcher_cat, runs_home, runs_away = gamelogs.loc[row, ['Date', 'Team_home', 'Team_away', 'OppStart_home', 'Categorization_home', 'R_home', 'R_away']]
        ### GET GAME RESULT ### 
        #game_outcome = (runs_home>runs_away).astype(int)
        ### GET GAME ODDS ###
        try:
            home_odds = odds.loc[(odds['Date']==date)&(odds['Home Team']==team)&(odds['Away Team']==opp), 'Home W Odds'].to_list()[0]
            away_odds = odds.loc[(odds['Date']==date)&(odds['Home Team']==team)&(odds['Away Team']==opp), 'Away W Odds'].to_list()[0]
        except:
            print('No odds for game on ', date, ' for ', team, ' - ', opp)
        ### GET GAMES TEAM HAD AGAINST SPECIFIC PITCHER THIS SEASON ###
        against_pitcher_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['OppStart_home']==pitcher_home)), 
                                col_home]

        against_pitcher_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['OppStart_away']==pitcher_home)&(gamelogs['Team_away']==team)), 
                                col_away]
        against_pitcher_away.columns = col_home
        against_pitcher = pd.concat([against_pitcher_away, against_pitcher_home])
        ### GET GAMES TEAM HAD AGAINST PITCHER CATEGORY THIS SEASON ###
        against_pitcher_cat_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Categorization_home']==pitcher_cat_home)&(gamelogs['OppStart_home']!=pitcher_home)),
                                    col_home]
        
        against_pitcher_cat_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Categorization_away']==pitcher_cat_home)&(gamelogs['OppStart_away']!=pitcher_home)&(gamelogs['Team_away']==team)),
                                    col_away]
        against_pitcher_cat_away.columns = col_home
        against_pitcher_cat = pd.concat([against_pitcher_cat_away, against_pitcher_cat_home])
        # against_pitcher_cat = against_pitcher_cat[-5:]
        ### GET GAMES TEAM HAD AGAINST OPPOSING TEAM THIS SEASON ###
        against_team_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==team)&(gamelogs['Team_away']==opp)&(gamelogs['OppStart_home']!=pitcher_home)&(gamelogs['Categorization_home']!=pitcher_cat_home)),
                                col_home]
        
        against_team_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&(((gamelogs['Team_home']==opp)&(gamelogs['Team_away']==team)&(gamelogs['OppStart_away']!=pitcher_home)&(gamelogs['Categorization_away']!=pitcher_cat_home))),
                                col_away]
        against_team_away.columns = col_home
        against_team = pd.concat([against_team_away, against_team_home])
        # against_team = against_team[-5:]
        
        home_team_last = []
        if len(against_pitcher)+len(against_pitcher_cat)+len(against_team)>3:
            home_team_last = pd.concat([against_pitcher, against_pitcher_cat, against_team]).groupby('Team_home', as_index=False).mean().drop('Team_home',axis=1)
            home_team_last.loc[~home_team_last['AB_home'].isna(), 'BA_home'] = np.round(home_team_last.loc[~home_team_last['AB_home'].isna(), 'H_home']/home_team_last.loc[~home_team_last['AB_home'].isna(), 'AB_home'],3)
            home_team_last.loc[~home_team_last['PA_home'].isna(), 'OBP_home'] = np.round((home_team_last.loc[~home_team_last['PA_home'].isna(), 'H_home']+home_team_last.loc[~home_team_last['PA_home'].isna(), 'BB_home'])\
                                                                                /home_team_last.loc[~home_team_last['PA_home'].isna(), 'PA_home'],3)
            home_team_last.loc[~home_team_last['AB_home'].isna(), 'SLG_home'] = np.round((home_team_last.loc[~home_team_last['AB_home'].isna(), '1B_home']+2*home_team_last.loc[~home_team_last['AB_home'].isna(), '2B_home']+\
                                                                                3*home_team_last.loc[~home_team_last['AB_home'].isna(), '3B_home']+4*home_team_last.loc[~home_team_last['AB_home'].isna(), 'HR_home'])\
                                                                                /home_team_last.loc[~home_team_last['AB_home'].isna(), 'AB_home'],3)
            home_team_last['BP_home'] = bullpens.loc[(bullpens['Team']==team)&(bullpens['Season']==date.year), 'Bullpen_Rating'].to_list()[0]
        ##########################################################################################################################################################################################################
        ### GETTING AWAY TEAM LAST GAMES STATS ###
        # date, team, opp, pitcher, pitcher_cat, runs_home, runs_away = gamelogs.loc[row, ['Date', 'Team_away', 'Team_home', 'OppStart_away', 'Categorization_away', 'R_home', 'R_away']]
        ### GET GAMES TEAM HAD AGAINST SPECIFIC PITCHER THIS SEASON ###
        against_pitcher_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==opp)&(gamelogs['OppStart_home']==pitcher_away)), 
                                col_home]

        against_pitcher_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['OppStart_away']==pitcher_away)&(gamelogs['Team_away']==opp)), 
                                col_away]
        against_pitcher_away.columns = col_home
        against_pitcher = pd.concat([against_pitcher_away, against_pitcher_home])
        ### GET GAMES TEAM HAD AGAINST PITCHER CATEGORY THIS SEASON ###
        against_pitcher_cat_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==opp)&(gamelogs['Categorization_home']==pitcher_cat_away)&(gamelogs['OppStart_home']!=pitcher_away)),
                                    col_home]
        
        against_pitcher_cat_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Categorization_away']==pitcher_cat_away)&(gamelogs['OppStart_away']!=pitcher_away)&(gamelogs['Team_away']==opp)),
                                    col_away]
        against_pitcher_cat_away.columns = col_home
        against_pitcher_cat = pd.concat([against_pitcher_cat_away, against_pitcher_cat_home])
        # against_pitcher_cat = against_pitcher_cat[-5:]
        ### GET GAMES TEAM HAD AGAINST OPPOSING TEAM THIS SEASON ###
        against_team_home = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&((gamelogs['Team_home']==opp)&(gamelogs['Team_away']==team)&(gamelogs['OppStart_home']!=pitcher_away)&(gamelogs['Categorization_home']!=pitcher_cat_away)),
                                col_home]
        
        against_team_away = gamelogs.loc[(gamelogs['Date'].dt.year==date.year)&(gamelogs['Date']<date)&(((gamelogs['Team_home']==team)&(gamelogs['Team_away']==opp)&(gamelogs['OppStart_away']!=pitcher_away)&(gamelogs['Categorization_away']!=pitcher_cat_away))),
                               col_away]
        against_team_away.columns = col_home
        against_team = pd.concat([against_team_away, against_team_home])
        # against_team = against_team[-5:]
        
        away_team_last = []
        if len(against_pitcher)+len(against_pitcher_cat)+len(against_team)>3:
            away_team_last = pd.concat([against_pitcher, against_pitcher_cat, against_team]).groupby('Team_home', as_index=False).mean().drop('Team_home',axis=1)
            away_team_last.loc[~away_team_last['AB_home'].isna(), 'BA_home'] = np.round(away_team_last.loc[~away_team_last['AB_home'].isna(), 'H_home']/away_team_last.loc[~away_team_last['AB_home'].isna(), 'AB_home'],3)
            away_team_last.loc[~away_team_last['PA_home'].isna(), 'OBP_home'] = np.round((away_team_last.loc[~away_team_last['PA_home'].isna(), 'H_home']+away_team_last.loc[~away_team_last['PA_home'].isna(), 'BB_home'])\
                                                                                /away_team_last.loc[~away_team_last['PA_home'].isna(), 'PA_home'],3)
            away_team_last.loc[~away_team_last['AB_home'].isna(), 'SLG_home'] = np.round((away_team_last.loc[~away_team_last['AB_home'].isna(), '1B_home']+2*away_team_last.loc[~away_team_last['AB_home'].isna(), '2B_home']+\
                                                                                3*away_team_last.loc[~away_team_last['AB_home'].isna(), '3B_home']+4*away_team_last.loc[~away_team_last['AB_home'].isna(), 'HR_home'])\
                                                                                /away_team_last.loc[~away_team_last['AB_home'].isna(), 'AB_home'],3)
            away_team_last['BP_home'] = bullpens.loc[(bullpens['Team']==opp)&(bullpens['Season']==date.year), 'Bullpen_Rating'].to_list()[0]

        # print(row, ':', opp, len(home_team_last), team, len(away_team_last))
        if (len(home_team_last)!=0) & (len(away_team_last)!=0):
            if (False in np.isnan(home_team_last.to_numpy())) & (False in np.isnan(away_team_last.to_numpy())) & (~np.isnan(home_odds)) & (~np.isnan(away_odds)):
                #game_results.append(game_outcome)
                game_stats.append(np.round(pd.concat([home_team_last, away_team_last], axis=1),3).to_numpy())
                all_home_odds.append(home_odds)
                all_away_odds.append(away_odds)
                new_results.append(results[i])
        
    return game_stats, all_home_odds, all_away_odds, new_results

In [ ]:
### PULLING ALL STATS FOR GAMELOGS ###
odds = pd.read_csv(os.path.join(data_path, "odds_list.csv"))
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Home Team'], odds['Away Team'] = odds['Home Team'].apply(lambda x: teams_abbr[x]), odds['Away Team'].apply(lambda x: teams_abbr[x])

# bullpens = pd.read_csv(os.path.join(data_path, "bullpens.csv"))
data_pull = BaseballScrapingModule.BaseballScraper()

gamelogs = structureGames()

game_stats, game_results, all_home_odds, all_away_odds, all_home_teams, all_away_teams, dates = getArraysUpToDate(odds, gamelogs, pd.to_datetime('today'), data_pull)

Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for season 2023 from FanGraphs...
Loading CSV data into a DataFrame...
Fetching CSV data for

In [17]:
#write output
np_game_stats, np_game_results = np.array(game_stats).reshape([len(game_stats), len(game_stats[0][0])]), np.array(game_results)
output = pd.DataFrame(np_game_stats)
output.columns = ['PA_home','AB_home','R_home','H_home','1B_home','2B_home','3B_home','HR_home','BB_home','SO_home','HBP_home','SH_home','SB_home','CS_home','LOB_home','BA_home','OBP_home','SLG_home', 'BP_home',
                  'PA_away','AB_away','R_away','H_away','1B_away','2B_away','3B_away','HR_away','BB_away','SO_away','HBP_away','SH_away','SB_away','CS_away','LOB_away','BA_away','OBP_away','SLG_away', 'BP_away']
output['Result'] = np_game_results
output.to_csv(os.path.join(data_path, 'compiled_stats.csv'), index=False)

In [24]:
### PULLING STATS FOR A SINGLE GAME ###
odds = pd.read_csv(os.path.join(data_path, "odds_list.csv"))
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Home Team'], odds['Away Team'] = odds['Home Team'].apply(lambda x: teams_abbr[x]), odds['Away Team'].apply(lambda x: teams_abbr[x])

gamelogs = structureGames()

date, teams, opps = pd.to_datetime('2023-06-11'), ['NYY'], ['BOS']
pitcher_homes, pitcher_cat_homes, pitcher_aways, pitcher_cat_aways = ['B.Bello'], [5.0], ['C.Schmidt'], [3.0] 

game_stats, all_home_odds, all_away_odds = getArraysForDate(odds, gamelogs, date, teams, opps, pitcher_homes, pitcher_cat_homes, pitcher_aways, pitcher_cat_aways)

In [9]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

def calculateWinnings(df, acc_balance, wins, bets_made, parlays_made, parlays_won):

    df['Parlay'] = 0

    possible_parlays = [team for team in powerset(df['HomeTeam']) if len(team)>1]
    parlay = pd.DataFrame()
    for i in possible_parlays:
        temp_parlay = df.loc[df['HomeTeam'].isin(i)]
        temp_parlay = temp_parlay.groupby('Date', as_index=False).agg({'HomeTeam':lambda x:list(x), 'AwayTeam':lambda x:list(x), 'Result':lambda x:list(x), 
                                                                    'Prediction':lambda x:list(x), 'Prob':'prod', 'Odds':'prod'})
        temp_parlay['Prob'], temp_parlay['Odds'] = round(temp_parlay['Prob'],4), round(temp_parlay['Odds'],2)
        if temp_parlay['Prob'].to_list()[0]>=0.5:
            parlay = pd.concat([parlay, temp_parlay])
            parlay['Parlay'] = 1

    df = pd.concat([df, parlay])

    df['kelly'] = round(df['Prob'] - (1-df['Prob'])/(df['Odds']-1),2)
    df['frac_kelly'] = round(df['kelly']/2,2)

    df = df.loc[df['frac_kelly']>=0.05]

    while sum(df['frac_kelly']) >=0.7:
        df['frac_kelly'] = round(df['frac_kelly']/2,2)

    df.sort_values('Prob', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)

    if len(df)==0:
        print('No bets made.')
    else:
        for i in range(len(df)):
            df.loc[i, 'WagerAmt'] = round(df.loc[i, 'frac_kelly']*acc_balance,2)
            acc_balance -= round(df.loc[i, 'frac_kelly']*acc_balance,2)

        df['Winnings'] = 0
        df.loc[df['Result']==df['Prediction'], 'Winnings'] = round(df.loc[df['Result']==df['Prediction'], 'WagerAmt']*df.loc[df['Result']==df['Prediction'], 'Odds'],2)

        df['Profit'] = round(df['Winnings'] - df['WagerAmt'],2)

        acc_balance += round(df['Winnings'].sum(),2)
        wins += sum((df['Prediction']==df['Result']).astype(int))
        bets_made += len(df)
        parlays_made += len(df.loc[df['Parlay']==1])
        parlays_won += len(df.loc[(df['Prediction']==df['Result'])&(df['Parlay']==1)])

    return round(acc_balance,2), wins, bets_made

In [ ]:
#set balance
acc_balance, wins, bets_made, parlays_made, parlays_won = 1000, 0, 0, 0, 0
#set seed for recreating
# random_seed = 54
# np.random.seed(random_seed)
# random.seed(random_seed)
### WHEN NOT SIMULATING, WE WILL NEED TO PULL FROM THE DAYS SCHEDULE ###
### BELIEVE I KEEP ORIGINAL GETARRAYS FUNCTION TO GET THE AGGREGATES UP TO THE GIVEN DAY FOR MODEL TRAINING. THEN RUN THE NEW GETARRAY TO GET THE AGGREGATED STATS FOR PREDICTING ###
odds = pd.read_csv(os.path.join(data_path, "odds_list.csv"))
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Home Team'], odds['Away Team'] = odds['Home Team'].apply(lambda x: teams_abbr[x]), odds['Away Team'].apply(lambda x: teams_abbr[x])

data_pull = BaseballScrapingModule.BaseballScraper()

# bullpens = pd.read_csv(os.path.join(data_path, "bullpens.csv"))

gamelogs = structureGames()

simulation_dates = sorted(list(set(gamelogs.loc[gamelogs['Date'].dt.year==2024, 'Date'].to_list())))[12:]

for date in simulation_dates:
    print(date)
# for date in [pd.to_datetime('2024-06-28'), pd.to_datetime('2024-06-29'), pd.to_datetime('2024-06-30'),pd.to_datetime('2024-07-01'), pd.to_datetime('2024-07-02'),
#              pd.to_datetime('2024-07-03'), pd.to_datetime('2024-07-04'), pd.to_datetime('2024-07-05'),pd.to_datetime('2024-07-06'), pd.to_datetime('2024-07-07')]:
    if acc_balance < 5:
        print("You're broke.")
        break
    ### GET ALL PREVIOUS DATA TO TRAIN ON ###
    game_stats, game_results, all_home_odds, all_away_odds, all_home_teams, all_away_teams, dates = getArraysUpToDate(odds, gamelogs, date, data_pull)
    game_stats, game_results = np.array(game_stats).reshape([len(game_stats), len(game_stats[0][0])]), np.array(game_results)

    #gradientboost
    # gbc = GradientBoostingClassifier(criterion='squared_error',n_estimators=100, learning_rate=0.15, max_depth=75).fit(game_stats, game_results)
    #randomforest
    # forest = RandomForestClassifier(criterion='entropy',max_depth=10,min_samples_split= 10,n_estimators= 200).fit(game_stats, game_results)
    #svm
    svm = SVC(C=10, gamma='scale', kernel='linear', probability=True).fit(game_stats, game_results)
    #xgboost
    params = {'num_class':2,
            'learning_rate': 0.01,
            'max_depth': 3,
            'n_estimators': 200,
            'subsample': 0.8,
            'eval_metric': 'logloss'}
    bst = xgb.XGBClassifier(objective='multi:softmax', **params).fit(game_stats, game_results)
    ### GET TODAY'S AGG STATS TO MAKE PREDICTION FROM TRAINED MODEL ###
    todays_game = gamelogs.loc[gamelogs['Date']==date]
    teams, opps = todays_game['Team_home'].to_list(), todays_game['Team_away'].to_list()
    pitcher_homes, pitcher_cat_homes = todays_game['OppStart_home'].to_list(), todays_game['Categorization_home'].to_list()
    pitcher_aways, pitcher_cat_aways = todays_game['OppStart_away'].to_list(), todays_game['Categorization_away'].to_list()

    results = todays_game['Result'].to_numpy()

    game_stats, all_home_odds, all_away_odds, new_results = getArraysForDate(odds, gamelogs, date, teams, opps, pitcher_homes, pitcher_cat_homes, pitcher_aways, pitcher_cat_aways, results,data_pull)
    if len(game_stats)==0:
        print(f"Not enough data, skipping {date}.")
        continue
    game_stats, new_results = np.array(game_stats).reshape([len(game_stats), len(game_stats[0][0])]), np.array(new_results)
    print('# Games: ', len(new_results), '\n')
    #gradient predict
    # grad_pred = gbc.predict(game_stats)
    # grad_pred_probs = gbc.predict_proba(game_stats)[:,1]
    # print('Gradient', ': ', accuracy_score(new_results, grad_pred), ': ', sum(grad_pred)/len(grad_pred), '\n') 
    #forest predict
    # for_pred = forest.predict(game_stats)
    # for_pred_probs = forest.predict_proba(game_stats)[:,1]
    # print('Forest: ', accuracy_score(new_results, for_pred),'', sum(for_pred)/len(for_pred), '\n')
    #svm predict
    svm_pred = svm.predict(game_stats)
    svm_pred_probs = svm.predict_proba(game_stats)[:,1]
    print('SVM: ', accuracy_score(new_results, svm_pred),'', sum(svm_pred)/len(svm_pred), '\n')
    #xg predict
    xg_pred = bst.predict(game_stats)
    xg_pred_probs = bst.predict_proba(game_stats)[:,1]
    print('XG: ', accuracy_score(new_results, xg_pred),'', sum(xg_pred)/len(xg_pred), '\n')
    #combine predictions
    # bets = for_pred  +  xg_pred + svm_pred
    bets = xg_pred + svm_pred
    teams_new, opps_new, results_new, predictions, pred_probs, home_odds_new, away_odds_new = [], [], [], [], [], [], []
    for i in range(len(bets)):
        if (bets[i]==2) | (bets[i]==0):
            teams_new.append(teams[i])
            opps_new.append(opps[i])
            results_new.append(new_results[i])
            predictions.append(bets[i]//2)
            pred_probs.append(round((xg_pred_probs[i]+svm_pred_probs[i])/2, 4))
            home_odds_new.append(all_home_odds[i])
            away_odds_new.append(all_away_odds[i])

    ### accuracy of all three combined
    if len(predictions)>0:
        print('Combined: ', accuracy_score(results_new, predictions),'', sum(predictions)/len(predictions), ': ', len(predictions))

        df = pd.DataFrame({'Date':[date]*len(predictions), 'HomeTeam':teams_new, 'AwayTeam':opps_new, 'Result':results_new, 'Prediction':predictions, 'Prob':pred_probs,
                'HomeOdds':home_odds_new, 'AwayOdds':away_odds_new})
        df.loc[df['Prediction']==0, 'Prob'] = 1 - df.loc[df['Prediction']==0, 'Prob']
        df.loc[df['Prediction']==0, 'Odds'] = df.loc[df['Prediction']==0, 'AwayOdds']
        df.loc[df['Prediction']==1, 'Odds'] = df.loc[df['Prediction']==1, 'HomeOdds']
        df.drop(['HomeOdds', 'AwayOdds'],axis=1,inplace=True)

        acc_balance, wins, bets_made = calculateWinnings(df, acc_balance, wins, bets_made, parlays_made, parlays_won)
        print('Balance: ', acc_balance, '\n', 'Wins: ', wins, '\n','Bets: ',bets_made, '\n','Parlays Won - Made: ', parlays_won, ' - ', parlays_made)
    else:
        print('No agreement.')

2024-04-07 00:00:00
Fetching CSV data for season 2024 from FanGraphs...
Loading CSV data into a DataFrame...
Bullpens ranked. 



IndexError: list index out of range

In [12]:
bullpens

,Season,Team,ERA,WAR,K/9,SV,ERA_norm,K9_norm,SV_norm,WAR_norm,Bullpen_Rating
0,2024,CLE,2.57,7.7,9.39,53,1.000000,0.551471,0.941176,1.000000,0.8732
3,2024,MIL,3.11,5.9,8.81,53,0.809859,0.338235,0.941176,0.823529,0.7282
2,2024,ATL,3.32,6.1,9.92,40,0.735915,0.746324,0.558824,0.843137,0.7210
12,2024,NYM,4.03,3.8,10.61,39,0.485915,1.000000,0.529412,0.617647,0.6582
6,2024,SDP,3.78,5.5,9.38,44,0.573944,0.547794,0.676471,0.784314,0.6456
...,...,...,...,...,...,...,...,...,...,...,...
21,2024,ARI,4.41,2.3,8.23,38,0.352113,0.125000,0.500000,0.470588,0.3619
26,2024,LAA,3.99,1.2,8.28,35,0.500000,0.143382,0.411765,0.362745,0.3545
27,2024,CHW,4.73,0.6,9.24,21,0.239437,0.496324,0.000000,0.303922,0.2599
28,2024,COL,5.41,-0.2,7.89,37,0.000000,0.000000,0.470588,0.225490,0.1740


In [10]:
np_game_stats, np_game_results = np.array(game_stats).reshape([len(game_stats), len(game_stats[0][0])]), np.array(game_results)
# min_max_scaler = preprocessing.MinMaxScaler()
# game_stats = min_max_scaler.fit_transform(game_stats)

stats_train, stats_test, results_train, results_test, h_odds_train, h_odds_test, a_odds_train, a_odds_test, \
    h_team_train, h_team_test, a_team_train, a_team_test = train_test_split(np_game_stats, np_game_results, all_home_odds, all_away_odds, all_home_teams, all_away_teams, test_size=0.2) 

# stats_train, stats_test = np.array(stats_train).reshape([len(stats_train), len(game_stats[0][0])]), np.array(stats_test).reshape([len(stats_test), len(game_stats[0][0])])
# results_train, results_test = np.array(results_train), np.array(results_test)

In [ ]:
### last best models ###
#gradientboost
# gbc = GradientBoostingClassifier(n_estimators=150, learning_rate=0.01, max_depth=5, random_state=40).fit(stats_train, results_train)
gbc = GradientBoostingClassifier(random_state=40, criterion='squared_error',n_estimators=100, learning_rate=0.15, max_depth=75).fit(stats_train, results_train)
grad_pred = gbc.predict(stats_test)
grad_pred_probs = gbc.predict_proba(stats_test)
print('Gradient: ', accuracy_score(results_test, grad_pred), '', sum(grad_pred)/len(grad_pred), '\n')
#randomforest
# forest = RandomForestClassifier(random_state=40, n_estimators=100, criterion='log_loss', min_samples_split=2, min_samples_leaf=3, min_weight_fraction_leaf=0.2).fit(stats_train, results_train)
forest = RandomForestClassifier(random_state=40, n_estimators=120, criterion='entropy', min_weight_fraction_leaf=0.1, max_depth=100).fit(stats_train, results_train)
for_pred = forest.predict(stats_test)
for_pred_probs = forest.predict_proba(stats_test)
print('Forest: ', accuracy_score(results_test, for_pred),'', sum(for_pred)/len(for_pred), '\n')
#neural
clf = MLPClassifier(random_state=40, hidden_layer_sizes=(100, 100), activation='tanh', solver='adam', alpha=0.001, learning_rate='constant', 
                    learning_rate_init=0.01).fit(stats_train, results_train)
neural_pred = clf.predict(stats_test)
neural_pred_probs = clf.predict_proba(stats_test)
print('Neural: ', accuracy_score(results_test, neural_pred),'', sum(neural_pred)/len(neural_pred), '\n')
#svm
# svm = SVC(C=2, kernel='sigmoid', gamma='scale', tol=0.1, random_state=40).fit(stats_train, results_train)
svm = SVC(random_state=40, C=0.5, kernel='linear',gamma='scale', tol=0.1, probability=True).fit(stats_train, results_train)
svm_pred = svm.predict(stats_test)
svm_pred_probs = svm.predict_proba(stats_test)[:,1]
print('SVM: ', accuracy_score(results_test, svm_pred),'', sum(svm_pred)/len(svm_pred), '\n')
#xgboost
params = {'booster':'gbtree',
'num_class': 2, 
'max_depth': 4,  
'learning_rate': 0.2,  
'eval_metric': 'mlogloss',
'lambda':1.2}
bst = xgb.XGBClassifier(objective='multi:softmax', **params).fit(stats_train, results_train)
xg_pred = bst.predict(stats_test)
xg_pred_probs = bst.predict_proba(stats_test)[:,1]
print('XG: ', accuracy_score(results_test, xg_pred),'', sum(xg_pred)/len(xg_pred), '\n')

Gradient:  0.5273556231003039  0.5106382978723404 

Forest:  0.560790273556231  0.7355623100303952 

Neural:  0.5212765957446809  1.0 

SVM:  0.5516717325227963  0.5987841945288754 

XG:  0.5349544072948328  0.5972644376899696 



In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Load the dataset
data = pd.read_csv(os.path.join(data_path, 'compiled_stats.csv'))

# Prepare the feature set and target variable
X = data.drop(columns=['Result'])
y = data['Result']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models and their hyperparameters for GridSearchCV
param_grids = {
    "Decision Tree": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {
            "criterion": ["gini", "entropy"],
            "max_depth": [5, 10, 15, None],
            "min_samples_split": [2, 5, 10]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [10, 20, None],
            "min_samples_split": [2, 5, 10],
            "criterion": ["gini", "entropy"]
        }
    },
    "K-Nearest Neighbors": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7, 9],
            "weights": ["uniform", "distance"],
            "metric": ["euclidean", "manhattan"]
        }
    },
    "Naive Bayes": {
        "model": GaussianNB(),
        "params": {}  # Naive Bayes does not have significant tunable hyperparameters
    },
    "SVM": {
        "model": SVC(random_state=42),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf", "poly"],
            "gamma": ["scale", "auto"]
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "learning_rate": [0.01, 0.1, 0.2],
            "max_depth": [3, 5, 10]
        }
    },
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "learning_rate": [0.01, 0.1, 0.2],
            "max_depth": [3, 5, 10],
            "subsample": [0.8, 1.0]
        }
    }
}

# Perform GridSearchCV and collect results
results = []
best_models = {}

for name, config in param_grids.items():
    model = config["model"]
    param_grid = config["params"]

    if param_grid:  # Apply grid search if parameters are provided
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_models[name] = best_model
    else:  # No grid search for models without significant parameters (Naive Bayes)
        best_model = model
        best_model.fit(X_train, y_train)
        best_params = "Default parameters used"
        best_models[name] = best_model

    # Predict on the test set and compute accuracy
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append({"Model": name, "Accuracy": acc, "Best Params": best_params})

# Display the results
results_df = pd.DataFrame(results)
print("Model Accuracy Comparison with Hyperparameter Tuning:")
print(results_df)

c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [18:04:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Model Accuracy Comparison with Hyperparameter Tuning:
                 Model  Accuracy  \
0        Decision Tree  0.541033   
1        Random Forest  0.563830   
2  K-Nearest Neighbors  0.528875   
3          Naive Bayes  0.550152   
4                  SVM  0.569909   
5    Gradient Boosting  0.554711   
6              XGBoost  0.577508   

                                         Best Params  
0  {'criterion': 'entropy', 'max_depth': 5, 'min_...  
1  {'criterion': 'entropy', 'max_depth': 10, 'min...  
2  {'metric': 'manhattan', 'n_neighbors': 3, 'wei...  
3                            Default parameters used  
4    {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}  
5  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
6  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  


In [ ]:
XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

,Model,Accuracy,Best Params
0,Decision Tree,0.541033,"{'criterion': 'entropy', 'max_depth': 5, 'min_..."
1,Random Forest,0.563830,"{'criterion': 'entropy', 'max_depth': 10, 'min..."
2,K-Nearest Neighbors,0.528875,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,Naive Bayes,0.550152,Default parameters used
4,SVM,0.569909,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}"
5,Gradient Boosting,0.554711,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est..."
6,XGBoost,0.577508,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est..."


In [39]:
params = {'num_class':2,
            'learning_rate': 0.01,
            'max_depth': 3,
            'n_estimators': 200,
            'subsample': 0.8,
            'eval_metric': 'logloss'}
bst = xgb.XGBClassifier(use_label_encoder=False,objective='multi:softmax', **params, random_state=42).fit(X_train, y_train)

c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [18:23:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [40]:
pred = bst.predict(X_test)
accuracy_score(y_test, pred)

0.5820668693009119